## Code to convert the GTSRB dataset to 32x32 pixel, and store it besides original dataset.

In [ ]:
import matplotlib.pyplot as plt
from gtrsb_loader.download_gtsrb import download_gtsrb

from skimage import data
from skimage.transform import resize
from PIL import Image

import numpy as np
import pandas as pd

import sys
import os
import fnmatch

In [ ]:
absolute_path = download_gtsrb()

In [ ]:
new_path = absolute_path.replace('GTSRB', 'GTSRB_32x32')
if os.path.isdir(new_path):
    print("GTSRB_32x32 dataset is in place, you're fine.")
    print(new_path)

In [ ]:
ppm_filelist = []
csv_filelist = []

for root, dirs, files in os.walk(absolute_path):
    ppm_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.ppm')]
    csv_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.csv')]

In [ ]:
print(ppm_filelist[0])

## process ppm files into new folder

In [ ]:
for idx, filepath in enumerate(ppm_filelist):
    pic = plt.imread(filepath)
    resized = resize(pic, (32, 32), mode='edge')
    im = Image.fromarray(np.uint8(resized * 255))
    new_path = filepath.replace('GTSRB', 'GTSRB_32x32')
    directory = os.path.split(new_path)[0]
    if not os.path.exists(directory):
        os.makedirs(directory) 
    im.save(new_path)
    if idx % 100 == 0: sys.stdout.write('\r' + str(idx) + "/39000" +" pictures processed")

## process .csv files into new folder

In [ ]:
def transform_csv(df: pd.DataFrame) -> pd.DataFrame:
    columns=['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId']
    matrix = df.as_matrix(columns=columns)
    matrix.shape
    matrix2 = np.zeros(matrix.shape)
    for i, row in enumerate(matrix):
        xfac=32/row[0]
        yfac=32/row[1]
        matrix2[i] =[32,32,round(row[2]*xfac),round(row[3]*yfac),round(row[4]*xfac),round(row[5]*yfac), row[6]]
    matrix2 = matrix2.astype(int)
    df2 = pd.DataFrame(matrix2, columns=columns)
    df2['Filename'] = df['Filename']
    df2 = df2[['Filename'] + columns]
    return df2

In [ ]:
for filepath in csv_filelist:
    df = pd.read_csv(filepath, sep=';')
    df2 = transform_csv(df)
    new_path = filepath.replace('GTSRB', 'GTSRB_32x32')
    df2.to_csv(new_path, sep=';', index=False)